In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from pathlib import Path
from glob import glob
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
from xorca.lib import load_xorca_dataset
# Personal modification to xorca
from __param__ import *
from tools_xorca import complete_dataset, plot_3D_averages
from calculate_xorca import average_3D, average_3D_list

In [2]:
#General definitions
config = 'ORCA1L75'
exp    = 'a13c'
data_path     = Path("/esnas/exp/ecearth/a13c/original_files/19500101/fc0/outputs")
data_path_aux = Path(coordinates_path+config)

#Data files
data_files_u  = list(sorted(data_path.glob(exp+"_1m_200[01]*grid_U.nc")))
data_files_v  = list(sorted(data_path.glob(exp+"_1m_200[01]*grid_V.nc")))
data_files_t  = list(sorted(data_path.glob(exp+"_1m_200[01]*grid_T.nc")))
data_files_sbc  = list(sorted(data_path.glob(exp+"_1m_200[01]*SBC.nc")))
aux_files_m   = list(sorted(data_path_aux.glob("mesh*")))
aux_files_s   = list(sorted(data_path_aux.glob("subbasins.nc")))
aux_files_b   = list(sorted(data_path_aux.glob("basin_mask_ORCA1_ece3.2_2017.nc4")))

data_files    = data_files_u+data_files_v+data_files_t
aux_files     = aux_files_m+aux_files_s+aux_files_b

In [3]:
#Actualy load the data
ds_xorca = load_xorca_dataset(data_files=data_files, aux_files=aux_files,
                              decode_cf=True, 
                              input_ds_chunks=input_chunks_ORCA1L75,
                              target_ds_chunks=target_chunks_ORCA1L75,
                              update_orca_variables=update_orca_variables)

In [4]:
#Add area and volume of the grid to the coordinates
ds_xorca = complete_dataset(ds_xorca)

In [13]:
def average_3D(ds,var,depths=[0]):
    """Calculate the volume weighted average of 'var' variable in ds
    Parameters
    ----------
    ds : xarray dataset
        A grid-aware dataset as produced by `xorca.lib.preprocess_orca`.
    var : name of the variable to average. Has to be included in ds and be 3D (+ time)
    depths : list of depths defining depth layers for averaging
    Returns
    -------
    ave : xarray data array
        A grid-aware data array with the averaged variable. 
        Time_series format 
    """

    if var in ds.variables.keys():
        v = ds[var]
        depths.append(10000) # do the calculation from last depth to the bottom
        dims  = update_orca_variables[var]['dims'][-3:]
        depth = [key for key in v.coords.keys() if key.startswith('depth')][0]
        vol   = [key for key in v.coords.keys() if key.endswith('vol')][0]
        arrays=[]; names_coord=[]
        if len(depths)>2:
            for depth1,depth2 in zip(depths,depths[1:]):
                condition = ((-v[depth]>depth1) & (-v[depth]<depth2))
                array = (v * v[vol] ).where(condition,drop=True).sum(dims) / (v[vol]).where(condition,drop=True).sum(dims)
                array = array.compute()
                arrays.append(array)
                if depth2==10000:
                    names_coord.append(str(depth1)+'-bottom')
                else:
                    names_coord.append(str(depth1)+'-'+str(depth2))
        #add the computation for the whole column too
        array = (v * v[vol] ).sum(dims) / (v[vol]).sum(dims)
        arrays.append(array)
        names_coord.append('0-bottom')
        depths.pop()
        ds_out = xr.concat(arrays,dim='depth_range')
        ds_out.name = var
        ds_out.coords['depth_range'] = names_coord
    else:
        print('Variable '+var+' is not in the dataset. Impossible to do average_3D. Please select another variable')

    return ds_out

In [14]:
def average_3D_list(ds_in,list_vars,depths=[0],outname='_3D_averages.nc'):
    """Calculate the volume weighted average for each var in list_vars, calling average_3D function
    """
    masks = [key for key in ds_in.coords.keys() if key.startswith('tmask')]
    averages = []
    for mask in masks:
        if mask != 'tmask':
            ds = ds_in.where(ds_in[mask],drop=True)    
        else:
            ds = ds_in
        for var in list_vars:
            print(mask,var,depths)
            ave = average_3D(ds,var,depths)
            ave.name = var+'_'+mask
            averages.append(ave)
        
    ds_out=[]
    for (i,var) in enumerate(list_vars):
        vars = [diag for diag in averages if diag.name.startswith(var)]
        ds_out.append(xr.concat(vars,dim='basins').rename(var+"_3D_ave"))
    ds_out = xr.merge(ds_out)
    ds_out.coords['basins'] = masks
    ds_out_yearly_rolling = ds_out.rolling(t=12, center=True).mean()
    ds_out=xr.concat((ds_out,ds_out_yearly_rolling),dim='data_type')
    ds_out.coords['data_type'] = ['monthly','yearly_rolling']
    ds_out.to_netcdf(outname)

    return ds_out

In [15]:
list_vars=['thetao','so']
depths=[0,100]
depths

[0, 100]

In [16]:
ds_out = average_3D_list(ds_xorca,list_vars,depths)
ds_out

tmask thetao [0, 100]
tmask so [0, 100]
tmaskatl thetao [0, 100]
tmaskatl so [0, 100]
tmaskind thetao [0, 100]
tmaskind so [0, 100]
tmaskpac thetao [0, 100]
tmaskpac so [0, 100]
tmaskgin thetao [0, 100]
tmaskgin so [0, 100]
tmasknna thetao [0, 100]
tmasknna so [0, 100]
tmaskmed thetao [0, 100]
tmaskmed so [0, 100]
tmasklab thetao [0, 100]
tmasklab so [0, 100]
tmaskwed thetao [0, 100]
tmaskwed so [0, 100]
tmaskarc thetao [0, 100]
tmaskarc so [0, 100]
tmasksoc thetao [0, 100]
tmasksoc so [0, 100]
tmaskindpac thetao [0, 100]
tmaskindpac so [0, 100]


<xarray.Dataset>
Dimensions:        (basins: 12, data_type: 2, depth_range: 3, t: 24)
Coordinates:
  * t              (t) datetime64[ns] 2000-01-16T12:00:00 ...
  * depth_range    (depth_range) <U10 '0-100' '100-bottom' '0-bottom'
  * basins         (basins) <U11 'tmask' 'tmaskatl' 'tmaskind' 'tmaskpac' ...
  * data_type      (data_type) <U14 'monthly' 'yearly_rolling'
Data variables:
    thetao_3D_ave  (data_type, basins, depth_range, t) float64 dask.array<shape=(2, 12, 3, 24), chunksize=(1, 1, 1, 12)>
    so_3D_ave      (data_type, basins, depth_range, t) float64 dask.array<shape=(2, 12, 3, 24), chunksize=(1, 1, 1, 12)>